
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
# from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw, ImageOps
from tqdm import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os
import copy
import torchvision

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.transforms.functional as tvF
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
from glob import glob
import albumentations as A
from albumentations.pytorch import ToTensorV2

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader, DatasetMapper, build_detection_train_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.checkpoint import DetectionCheckpointer
import detectron2.data.transforms as T
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, LVISEvaluator
setup_logger()

In [ ]:
sigmoid = nn.Sigmoid()

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# # You need to mount your google drive in order to load the data:
# from google.colab import drive
# drive.mount('/content/drive')
# # Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# # Also create an output directory for your files such as the trained models and the output images.

In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = './'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''
images_min_max_cords = {}
def get_detection_data(set_name, complete=False):
    global images_min_max_cords
    images_min_max_cords = {}
    data_dirs = '{}/data'.format(BASE_DIR)
    record = {}
    dataset_dicts = []
    if set_name == "train" or set_name == "val":
        processed_data_dirs = '{}/object_detection_data'.format(BASE_DIR)
        os.makedirs(os.path.join(processed_data_dirs, "train"), exist_ok=True)
        with open(os.path.join(data_dirs, "train.json")) as f:
            imgs_anns = json.load(f)
        sorted_imgs_anns = sorted(imgs_anns, key=lambda x:x['file_name'])
        i = 0
        previous_file_name = None
        record["file_name"] = os.path.join(data_dirs, "train", sorted_imgs_anns[0]["file_name"])
        record["image_id"] = sorted_imgs_anns[0]["image_id"]
        record['width'], record['height'] = Image.open(os.path.join(data_dirs, "train", sorted_imgs_anns[0]['file_name'])).size
        record['annotations'] = [{
                'bbox_mode': BoxMode.XYWH_ABS,
                "category_id": 0,
                "segmentation": sorted_imgs_anns[0]['segmentation'],
                "bbox": sorted_imgs_anns[0]['bbox']
            }]
        x, y, dx, dy = sorted_imgs_anns[0]['bbox']
        x, y, dx, dy = int(x), int(y), int(dx), int(dy)
        min_x_start, min_y_start, max_x_end, max_y_end = x, y, x+dx, y+dy
        while True:
            if i+1 < len(sorted_imgs_anns) and sorted_imgs_anns[i+1]["file_name"] == sorted_imgs_anns[i]["file_name"]:
                record['annotations'].append({
                    'bbox_mode': BoxMode.XYWH_ABS,
                    "category_id": 0,
                    "segmentation": sorted_imgs_anns[i+1]['segmentation'],
                    "bbox": sorted_imgs_anns[i+1]['bbox']
                })
                x, y, dx, dy = sorted_imgs_anns[i+1]['bbox']
                x, y, dx, dy = int(x), int(y), int(dx), int(dy)
                min_x_start = min(min_x_start, x)
                min_y_start = min(min_y_start, y)
                max_x_end = max(max_x_end, x+dx)
                max_y_end = max(max_y_end, y+dy)
                i += 1
                continue
            else:
                i += 1
#             processed_image_path = os.path.join(processed_data_dirs, "train", sorted_imgs_anns[i-1]["file_name"])
#             record["file_name"] = processed_image_path
#             if not os.path.isfile(processed_image_path):
#                 Image.fromarray(cv2.imread(os.path.join(data_dirs, "train", sorted_imgs_anns[i-1]["file_name"]))[min_y_start:max_y_end, min_x_start:max_x_end]).save(processed_image_path)
#             for annotation in record["annotations"]:
#                 x, y, dx, dy = annotation["bbox"]
#                 x, y, dx, dy = int(x), int(y), int(dx), int(dy)
#                 annotation["bbox"] = [x-min_x_start, y-min_y_start, dx, dy]
#                 for s in range(len(annotation["segmentation"])):
#                     for j in range(0, len(annotation["segmentation"][s]), 2):
#                         annotation["segmentation"][s][j] = annotation["segmentation"][s][j] - min_x_start
#                         annotation["segmentation"][s][j+1] = annotation["segmentation"][s][j+1] - min_y_start
#             images_min_max_cords[record["file_name"]] = [min_x_start, min_y_start, record['width'], record['height']]
#             record['width'], record['height'] = max_x_end - min_x_start, max_y_end - min_y_start
            dataset_dicts.append(record)
            record = {}
            if i == len(sorted_imgs_anns):
                break
            record["file_name"] = os.path.join(data_dirs, "train", sorted_imgs_anns[i]["file_name"])
            record["image_id"] = sorted_imgs_anns[i]["image_id"]
            record['width'], record['height'] = Image.open(os.path.join(data_dirs, "train", sorted_imgs_anns[i]['file_name'])).size
            record['annotations'] = [{
                    'bbox_mode': BoxMode.XYWH_ABS,
                    "category_id": 0,
                    "segmentation": sorted_imgs_anns[i]['segmentation'],
                    "bbox": sorted_imgs_anns[i]['bbox']
                }]
            x, y, dx, dy = sorted_imgs_anns[i]['bbox']
            x, y, dx, dy = int(x), int(y), int(dx), int(dy)
            min_x_start, min_y_start, max_x_end, max_y_end = x, y, x+dx, y+dy
        random.Random(4).shuffle(dataset_dicts)
#         random.shuffle(dataset_dicts)
#         if set_name == "train":
#             dataset_dicts = dataset_dicts[int(len(dataset_dicts)*0.15):]
        if set_name == "val":
            dataset_dicts = dataset_dicts[:int(len(dataset_dicts)*0.15)]
    elif set_name == "test":
        img_file_pathes = glob(os.path.join(data_dirs, 'test/*.png'))
        for idx, img_file_path in enumerate(img_file_pathes):
            img = Image.open(img_file_path)
            record = {}
            record["file_name"] = img_file_path
            record["image_id"] = idx
            record['width'], record['height'] = Image.open(img_file_path).size
            record['annotations'] = []
            dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
for d in ["train", "val", "test"]:
    DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
    MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
plane_metadata = MetadataCatalog.get("data_detection_train")

In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
dataset_dicts = get_detection_data("val")
# dataset_dicts = patched_dataset_dicts
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plane_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    plt.figure(figsize = (20,20))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()

### Set Configs

In [ ]:
models = {
    "R101-FPN": "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml",
    "R_50_FPN": "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml",#
    "R50-FPN": "COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml",
    "X101-FPN": "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml",
    "FAST_RCN": "COCO-Detection/fast_rcnn_R_50_FPN_1x.yaml",
    "R50_DC5": "COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_1x.yaml"
}

In [ ]:
!rm -rf output/object_detection/*

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/object_detection".format(BASE_DIR)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.merge_from_file(model_zoo.get_config_file(models["X101-FPN"]))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ("data_detection_val",)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(models["X101-FPN"])  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.


In [ ]:
class CostumTrainer(DefaultTrainer):
    def build_train_loader(cls, cfg):
        dataloader = build_detection_train_loader(
                        cfg,
                        mapper=DatasetMapper(cfg,is_train=True, augmentations=[
                            T.ResizeShortestEdge(short_edge_length=(600, 800), max_size=1333, sample_style='choice'),
                            T.RandomFlip(),
                            T.RandomBrightness(0.5, 1.5),
                        ])
                    )
        return dataloader

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
trainer = CostumTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3): 
# dataset_dicts = get_detection_data("train")
# for d in dataset_dicts: 
    print(d["file_name"])
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(20, 20))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
evaluator = COCOEvaluator("data_detection_val", output_dir="./output/object_detection/")
val_loader = build_detection_test_loader(cfg, "data_detection_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`


### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''


## Part 2: Semantic Segmentation

In [ ]:
!rm -rf output/semantic_segmentation/

In [ ]:
semantic_segmentation_output_dir = f"{BASE_DIR}/output/semantic_segmentation"
os.makedirs(semantic_segmentation_output_dir, exist_ok=True)

### Data Loader

In [ ]:
!rm -rf preprocessed_data/

In [ ]:
directories = {
    "train":{
        "images_dir": "./preprocessed_data/train/images",
        "masks_dir": "./preprocessed_data/train/masks",
    },
    "val":{
        "images_dir": "./preprocessed_data/val/images",
        "masks_dir": "./preprocessed_data/val/masks",
    },
}

In [ ]:
for dataset in directories:
    for path in directories[dataset]:
        os.makedirs(directories[dataset][path], exist_ok=True)

In [ ]:
for dataset in ["train", "val"]:
    if len(os.listdir(directories[dataset]["images_dir"])) > 0 and len(os.listdir(directories[dataset]["masks_dir"])) > 0:
        print(f"preprocessed data for {dataset} exists")
        continue
    print("preprocessing the data")
    my_data_list = DatasetCatalog.get("data_detection_{}".format(dataset))
    for data in tqdm(my_data_list):
        img = cv2.imread(data['file_name'])
        for idx, annotation in enumerate(data['annotations']):
            x, y, dx, dy = annotation['bbox']
            x, y, dx, dy = int(x), int(y), int(dx), int(dy)
#             obj_img = img.crop((x, y, x+dx, y+dy))
            obj_img = img[y:y+dx, x:x+dx]
            obj_x, obj_y, c = obj_img.shape
            obj_mask = np.zeros((obj_y, obj_x))
            polygon = []
            for i in range(0, len(annotation['segmentation'][0]), 2):
                s_x, s_y = annotation['segmentation'][0][i:i+2]
                polygon.append([s_x-x, s_y-y])

            contours = np.array(polygon)
            cv2.fillPoly(obj_mask, pts = [contours], color =(1,1,1))
            file_name = data['file_name'].split("/")[-1]
            Image.fromarray(obj_img).save(os.path.join(directories[dataset]["images_dir"], f"{idx}_{file_name}"))
            if len(np.array(obj_img).shape) == 2:
                print(data['file_name'])
            img_name = file_name.split(".")[0]
            np.save(os.path.join(directories[dataset]["masks_dir"], f"{idx}_{img_name}"), obj_mask)

In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
def get_instance_sample(data, idx, dataset="train"):
    file_name = data['file_name'].split("/")[-1]
    try:
#         obj_img = cv2.imread(os.path.join(directories[dataset]["images_dir"], f"{idx}_{file_name}"))
        obj_img = np.array(ImageOps.grayscale(Image.open(os.path.join(directories[dataset]["images_dir"], f"{idx}_{file_name}"))))
    except:
#         obj_img = cv2.imread(os.path.join(directories["val"]["images_dir"], f"{idx}_{file_name}"))
        obj_img = np.array(ImageOps.grayscale(Image.open(os.path.join(directories["val"]["images_dir"], f"{idx}_{file_name}"))))
    img_name = file_name.split(".")[0]
    try:
        obj_mask = np.load(os.path.join(directories[dataset]["masks_dir"], f"{idx}_{img_name}.npy"))
    except:
        obj_mask = np.load(os.path.join(directories["val"]["masks_dir"], f"{idx}_{img_name}.npy"))
    return obj_img, obj_mask

In [ ]:
img_size = (128, 128)

In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
    def __init__(self, set_name, data_list):
        if set_name == "train":
            self.transforms = A.Compose([
                A.Resize(img_size[0], img_size[1], cv2.INTER_NEAREST),
                A.ShiftScaleRotate(border_mode=cv2.BORDER_CONSTANT, 
                                      scale_limit=0.3,
                                      rotate_limit=(30, 30),
                                      p=0.75),
                A.Flip(),
                A.Normalize((0.5227), (0.2320)),
#                 A.Normalize([0.5309, 0.5279, 0.5051], [0.2326, 0.2342, 0.2327]),
                ToTensorV2(),
            ], p=1)
        else:
            self.transforms = A.Compose([
                A.Resize(img_size[0], img_size[1], cv2.INTER_NEAREST),
#                 A.Normalize([0.5309, 0.5279, 0.5051], [0.2326, 0.2342, 0.2327]),
                A.Normalize((0.5227), (0.2320)),
                ToTensorV2(),
            ], p=1)
        
        self.set_name = set_name
        self.data = data_list
        self.instance_map = []
        for i, d in enumerate(self.data):
            for j in range(len(d['annotations'])):
                self.instance_map.append([i,j])

    '''
    # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
    # make sure to use the correct length for the final training
    '''
    def __len__(self):
        return len(self.instance_map)

    def numpy_to_tensor(self, img, mask):
        if self.transforms is not None:
            img = self.transforms(img)
        img = torch.tensor(img, dtype=torch.float)
        mask = torch.tensor(mask, dtype=torch.float)
        return img, mask

    '''
    # Complete this part by using get_instance_sample function
    # make sure to resize the img and mask to a fixed size (for example 128*128)
    # you can use "interpolate" function of pytorch or "numpy.resize"
    # TODO: 5 lines
    '''
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        idx = self.instance_map[idx]
        data = self.data[idx[0]]
        img, mask = get_instance_sample(data, idx[1], self.set_name)
        transformed = self.transforms(image=img, mask=mask)
        img, mask = transformed['image'], transformed['mask']

        return img, torch.unsqueeze(mask, 0)

def get_plane_dataset(set_name='train', batch_size=32):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=8,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

In [ ]:
loader, dataset = get_plane_dataset(batch_size=512)

In [ ]:
def batch_mean_and_sd(loader):
    
    cnt = 0
    fst_moment = torch.empty(3)
    snd_moment = torch.empty(3)

    for images, _ in tqdm(loader):
        images = images / 255.0
        b, c, h, w = images.shape
        nb_pixels = b * h * w
        sum_ = torch.sum(images, dim=[0, 2, 3])
        sum_of_square = torch.sum(images ** 2,
                                  dim=[0, 2, 3])
        fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
        snd_moment = (cnt * snd_moment + sum_of_square) / (cnt + nb_pixels)
        cnt += nb_pixels

    mean, std = fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)        
    return mean,std

In [ ]:
mean, std = batch_mean_and_sd(loader)

In [ ]:
mean

In [ ]:
std

### Network

In [ ]:
class batchnorm_relu(nn.Module):
    def __init__(self, in_c):
        super().__init__()

        self.bn = nn.BatchNorm2d(in_c)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.bn(inputs)
        x = self.relu(x)
        return x

In [ ]:
class residual_block(nn.Module):
    def __init__(self, in_c, out_c, stride=1):
        super().__init__()

        """ Convolutional layer """
        self.b1 = batchnorm_relu(in_c)
        self.c1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, stride=stride)
        self.b2 = batchnorm_relu(out_c)
        self.c2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1, stride=1)

        """ Shortcut Connection (Identity Mapping) """
        self.s = nn.Conv2d(in_c, out_c, kernel_size=1, padding=0, stride=stride)

    def forward(self, inputs):
        x = self.b1(inputs)
        x = self.c1(x)
        x = self.b2(x)
        x = self.c2(x)
        s = self.s(inputs)

        skip = x + s
        return skip

In [ ]:
class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.r = residual_block(in_c+out_c, out_c)

    def forward(self, inputs, skip):
        x = self.upsample(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.r(x)
        return x

In [ ]:
class build_resunet(nn.Module):
    def __init__(self, out_sz):
        super().__init__()
        self.out_sz = out_sz
        """ Encoder 1 """
        self.c11 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.br1 = batchnorm_relu(64)
        self.c12 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.c13 = nn.Conv2d(1, 64, kernel_size=1, padding=0)

        """ Encoder 2 and 3 """
        self.r2 = residual_block(64, 128, stride=2)
        self.r3 = residual_block(128, 256, stride=2)

        """ Bridge """
        self.r4 = residual_block(256, 512, stride=2)

        """ Decoder """
        self.d1 = decoder_block(512, 256)
        self.d2 = decoder_block(256, 128)
        self.d3 = decoder_block(128, 64)

        """ Output """
        self.output = nn.Conv2d(64, 1, kernel_size=1, padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs):
        """ Encoder 1 """
        x = self.c11(inputs)
        x = self.br1(x)
        x = self.c12(x)
        s = self.c13(inputs)
        skip1 = x + s

        """ Encoder 2 and 3 """
        skip2 = self.r2(skip1)
        skip3 = self.r3(skip2)

        """ Bridge """
        b = self.r4(skip3)

        """ Decoder """
        d1 = self.d1(b, skip3)
        d2 = self.d2(d1, skip2)
        d3 = self.d3(d2, skip1)

        """ output """
        output = self.output(d3)
        out = F.interpolate(output, self.out_sz)
#         output = self.sigmoid(output)

        return output

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True, kernel=3):
        super(conv, self).__init__()
        if(activation):
            self.layer = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True)
            )
        else:
            self.layer = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel)  
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class Conv_Block(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Conv_Block, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            conv(out_ch, out_ch),
        )

    def forward(self, x):
        x = self.layer(x)
        return x

    
'''
# upsampling module equal to a upsample function followed by a conv module
'''

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
def crop(encoder_features, x):
    _, _, H, W = x.shape
    encoder_features   = torchvision.transforms.CenterCrop([H, W])(encoder_features)
    return encoder_features



class MyModel(nn.Module):
    def __init__(self, enc_chs=(1,64,128), dec_chs=(128,64), num_class=1, out_sz=(100, 100)):
        super(MyModel, self).__init__()
        self.enc_chs = enc_chs
        self.dec_chs = dec_chs
        self.out_sz = out_sz
        # Encoder
        self.enc_blocks = nn.ModuleList([Conv_Block(enc_chs[i], enc_chs[i+1]) for i in range(len(enc_chs)-1)])
        self.pool = nn.MaxPool2d(2)
        
        # Decoder
        self.dec_blocks = nn.ModuleList([Conv_Block(dec_chs[i], dec_chs[i+1]) for i in range(len(dec_chs)-1)]) 
        self.upconvs = nn.ModuleList([nn.ConvTranspose2d(dec_chs[i], dec_chs[i+1], 2, 2) for i in range(len(dec_chs)-1)])
        
        self.head = conv(dec_chs[-1], num_class, kernel=1)

    def forward(self, x):
        encoder_features = []
        for block in self.enc_blocks:
            x = block(x)
            encoder_features.append(x)
            x = self.pool(x)
        x, encoder_features = encoder_features[::-1][0], encoder_features[::-1][1:]
        for i in range(len(self.dec_chs)-1):
            x = self.upconvs[i](x)
            enc_ftrs = crop(encoder_features[i], x)
            x = torch.cat([x, enc_ftrs], dim=1)
            x = self.dec_blocks[i](x)
        out = self.head(x)
        out = F.interpolate(out, self.out_sz)
        return out

### Training

In [ ]:
!rm -rf output/semantic_segmentation/*

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''

# Set the hyperparameters
num_epochs = 40
batch_size = 32
learning_rate = 0.004
weight_decay = 1e-4

model = build_resunet(out_sz=img_size) # initialize the model
model = model.cuda() # move the model to GPU
loader, dataset = get_plane_dataset('train', batch_size) # initialize data_loader
val_loader, val_dataset = get_plane_dataset('val', batch_size) # initialize data_loader
# crit = nn.BCEWithLogitsLoss() # Define the loss function
# optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD
optim = torch.optim.Adam(model.parameters(), lr=learning_rate) # Initialize the optimizer as SGD
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optim, learning_rate, epochs=num_epochs, steps_per_epoch=len(dataset)//batch_size)
# optim = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-8, momentum=0.9)
# lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 'max', patience=2)  # goal: maximize Dice score
crit = DiceLoss()

# start the training procedure
train_losses = []
val_losses = []
train_ious = []
val_ious = []
best_val_loss = 10000
best_val_iou = 0
total_iou = 0
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    val_total_loss = 0
    model = model.train()
    progress_bar = tqdm(loader)
    data_count = 0
    for idx, (img, mask) in enumerate(progress_bar):
        progress_bar.set_description(f"Train Epoch {epoch}")
        img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
        mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
        pred = model(img)
        loss = crit(pred, mask)
        optim.zero_grad()
        loss.backward()
        optim.step()
        lr_scheduler.step()
        total_loss += loss.cpu().item()
        with torch.no_grad():
            pred_mask = torch.where(sigmoid(pred.detach())<=0.5, 0, 1)
            iou = torch.sum(torch.logical_and(mask, pred_mask)) / torch.sum(torch.logical_or(mask, pred_mask))
            total_iou = ((total_iou * data_count) + (img.shape[0] * iou)) / (data_count + img.shape[0])
        data_count += img.shape[0]
        train_ious.append(total_iou.item())
        progress_bar.set_postfix(
            {
                "loss": total_loss/(idx+1),
                "iou": total_iou.item()
            }
        )
#     torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))
    model = model.eval()
    progress_bar = tqdm(val_loader)
    data_count = 0
    for idx, (img, mask) in enumerate(progress_bar):
        progress_bar.set_description(f"Val Epoch {epoch}")
        img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
        mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
        pred = model(img)
        loss = crit(pred, mask)
        data_count += 1
        val_total_loss += loss.cpu().item()
        with torch.no_grad():
            pred_mask = torch.where(sigmoid(pred.detach())<=0.5, 0, 1)
            iou = torch.sum(torch.logical_and(mask, pred_mask)) / torch.sum(torch.logical_or(mask, pred_mask))
            total_iou = ((total_iou * data_count) + (img.shape[0] * iou)) / (data_count + img.shape[0])
        data_count += img.shape[0]
        val_ious.append(total_iou.item())
        progress_bar.set_postfix(
            {
                "loss": val_total_loss/(idx+1),
                "iou": total_iou.item()
            }
        )
    train_losses.append(total_loss/len(loader))
    val_losses.append(val_total_loss/len(val_loader))
    fig = plt.figure(figsize=(5, 5))
    fig.add_subplot(1, 1, 1)
    plt.plot(train_losses, label="train")
    plt.plot(val_losses, label="val")
    plt.legend()
    plt.show()
    if total_iou.item() > best_val_iou:
        best_val_iou = total_iou.item()
        torch.save(model.state_dict(), os.path.join(semantic_segmentation_output_dir, "best_segmentation_model.pth"))

'''
# Saving the final model
'''
torch.save(model.state_dict(), os.path.join(semantic_segmentation_output_dir, 'final_segmentation_model.pth'))


In [ ]:
fig = plt.figure(figsize=(5, 5))
fig.add_subplot(1, 1, 1)
plt.plot(train_losses, label="train")
plt.plot(val_losses, label="val")
plt.legend()

### Evaluation and Visualization

In [ ]:
model = build_resunet(out_sz=img_size).cuda()
model.load_state_dict(torch.load('{}/output/semantic_segmentation/best_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = build_resunet(out_sz=img_size).cuda()
model.load_state_dict(torch.load('{}/output/semantic_segmentation/best_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

total_iou = 0
data_count = 0
for (img, mask) in tqdm(loader):
    with torch.no_grad():
        img = img.cuda()
        mask = mask.cuda()
        pred = torch.where(sigmoid(model(img))<=0.5, 0, 1)
        iou = torch.sum(torch.logical_and(mask, pred)) / torch.sum(torch.logical_or(mask, pred))
        total_iou = ((total_iou * data_count) + (img.shape[0] * iou)) / (data_count + img.shape[0])
        data_count += img.shape[0]
    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    

print("\n #images: {}, Mean IoU: {}".format(data_count, total_iou))


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
loader, dataset = get_plane_dataset('val', 32)
model.eval()
img, mask = next(iter(loader))
with torch.no_grad():
    img = img.cuda()
    mask = mask
    pred = pred = torch.where(sigmoid(model(img))<=0.5, 0, 1).cpu()

In [ ]:
fig = plt.figure(figsize=(10, 10))
for i in range(3):
    fig.add_subplot(3, 3, (i*3)+1)
    plt.imshow(img[i, 0].cpu(), cmap="gray")
    plt.title('original_image')
    fig.add_subplot(3, 3, (i*3)+2)
    plt.imshow(mask[i, 0].cpu(), cmap="gray")
    plt.title('mask')
    fig.add_subplot(3, 3, (i*3)+3)
    plt.imshow(pred[i, 0].cpu(), cmap="gray")
    plt.title('prediction')
plt.show()

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''

transformation = A.Compose([
    A.Resize(img_size[0], img_size[1], cv2.INTER_NEAREST),
    A.Normalize((0.5256), (0.2275)),
    ToTensorV2(),
], p=1)

def get_prediction_mask(data, train_test, complete=True):
    global model
    predictor.model = predictor.model.eval()
    model = model.eval()
    img = cv2.imread(data['file_name'])
    outputs = predictor(img)
#     bboxes = outputs['instances'].pred_boxes.tensor
    bboxes = outputs['instances'].get_fields()['pred_boxes'].tensor[outputs['instances'].get_fields()['scores'] >= 0.9]
    min_x_start, min_y_start = 0, 0
#     if train_test == "train": 
#         min_x_start, min_y_start, width, height = images_min_max_cords[data['file_name']]
#     else:
    height, width = img.shape[:2]
    gt_mask = torch.zeros((height, width), dtype=torch.uint8)
    pred_mask = torch.zeros((height, width), dtype=torch.uint8)
    for idx, bbox in enumerate(bboxes):
        start_x, start_y, end_x, end_y = bbox
        start_x, start_y, end_x, end_y = int(start_x), int(start_y), int(end_x), int(end_y)
        obj_img = img[start_y:end_y, start_x:end_x]
        transformed = transformation(image=np.array(ImageOps.grayscale(Image.fromarray(obj_img))))
        obj_img = transformed['image']
        obj_img = obj_img.cuda()
        obj_pred_mask = torch.where(sigmoid(model(torch.unsqueeze(obj_img, 0)))<=0.5, 0, 1).cpu()
        pred_mask[min_y_start+start_y:min_y_start+end_y, min_x_start+start_x:min_x_start+end_x] = (idx+1) * F.interpolate(obj_pred_mask.type(torch.uint8), size=[end_y-start_y, end_x-start_x])
    for j in range(len(data['annotations'])):
        obj_img, obj_mask = get_instance_sample(data, j)
        x, y, dx, dy = data['annotations'][j]['bbox']
        x, y, dx, dy = int(x), int(y), int(dx), int(dy)
        obj_mask = np.zeros((dy, dx))
        polygon = []
        for i in range(0, len(data['annotations'][j]['segmentation'][0]), 2):
            s_x, s_y = data['annotations'][j]['segmentation'][0][i:i+2]
            polygon.append([s_x-x, s_y-y])

        contours = np.array(polygon)
        cv2.fillPoly(obj_mask, pts = [contours], color =(1,1,1))
        gt_mask[min_y_start+int(y):min_y_start+int(y+dy), min_x_start+int(x):min_x_start+int(x+dx)] = torch.tensor(obj_mask)
#     if train_test == "train":
#         img = cv2.imread(os.path.join(BASE_DIR, "data", "train", data['file_name'].split("/")[-1]))
    return img, gt_mask, pred_mask # gt_mask could be all zero when the ground truth is not given.


### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
fig = plt.figure(figsize=(50, 50))

dataset_dicts = get_detection_data("test")
for i, d in enumerate(random.sample(dataset_dicts, 3)):
    img, gt_mask, pred_mask = get_prediction_mask(d, "test")
    fig.add_subplot(3, 3, (i*3)+1)
    plt.imshow(img)
    plt.title('original_image')
    fig.add_subplot(3, 3, (i*3)+2)
    plt.imshow(gt_mask, cmap="rainbow")
    plt.title('mask')
    fig.add_subplot(3, 3, (i*3)+3)
    plt.imshow(pred_mask, cmap="rainbow")
    plt.title('prediction')
plt.show()


In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
        return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''
# dataset_dicts = get_detection_data("train")
preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
# my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
my_data_list = get_detection_data("train")
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample, "train")
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask.cuda()==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

'''
# Writing the predictions of the test set
'''
# my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
my_data_list = get_detection_data("test")
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample, "test")
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask.cuda()==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''

'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''
images_min_max_cords = {}
def get_detection_data(set_name, complete=False):
    global images_min_max_cords
    images_min_max_cords = {}
    data_dirs = '{}/data'.format(BASE_DIR)
    record = {}
    dataset_dicts = []
    if set_name == "train" or set_name == "val":
        processed_data_dirs = '{}/object_detection_data'.format(BASE_DIR)
        os.makedirs(os.path.join(processed_data_dirs, "train"), exist_ok=True)
        with open(os.path.join(data_dirs, "train.json")) as f:
            imgs_anns = json.load(f)
        sorted_imgs_anns = sorted(imgs_anns, key=lambda x:x['file_name'])
        i = 0
        previous_file_name = None
        record["file_name"] = os.path.join(data_dirs, "train", sorted_imgs_anns[0]["file_name"])
        record["image_id"] = sorted_imgs_anns[0]["image_id"]
        record['width'], record['height'] = Image.open(os.path.join(data_dirs, "train", sorted_imgs_anns[0]['file_name'])).size
        record['annotations'] = [{
                'bbox_mode': BoxMode.XYWH_ABS,
                "category_id": 0,
                "segmentation": sorted_imgs_anns[0]['segmentation'],
                "bbox": sorted_imgs_anns[0]['bbox']
            }]
        x, y, dx, dy = sorted_imgs_anns[0]['bbox']
        x, y, dx, dy = int(x), int(y), int(dx), int(dy)
        min_x_start, min_y_start, max_x_end, max_y_end = x, y, x+dx, y+dy
        while True:
            if i+1 < len(sorted_imgs_anns) and sorted_imgs_anns[i+1]["file_name"] == sorted_imgs_anns[i]["file_name"]:
                record['annotations'].append({
                    'bbox_mode': BoxMode.XYWH_ABS,
                    "category_id": 0,
                    "segmentation": sorted_imgs_anns[i+1]['segmentation'],
                    "bbox": sorted_imgs_anns[i+1]['bbox']
                })
                x, y, dx, dy = sorted_imgs_anns[i+1]['bbox']
                x, y, dx, dy = int(x), int(y), int(dx), int(dy)
                min_x_start = min(min_x_start, x)
                min_y_start = min(min_y_start, y)
                max_x_end = max(max_x_end, x+dx)
                max_y_end = max(max_y_end, y+dy)
                i += 1
                continue
            else:
                i += 1
            dataset_dicts.append(record)
            record = {}
            if i == len(sorted_imgs_anns):
                break
            record["file_name"] = os.path.join(data_dirs, "train", sorted_imgs_anns[i]["file_name"])
            record["image_id"] = sorted_imgs_anns[i]["image_id"]
            record['width'], record['height'] = Image.open(os.path.join(data_dirs, "train", sorted_imgs_anns[i]['file_name'])).size
            record['annotations'] = [{
                    'bbox_mode': BoxMode.XYWH_ABS,
                    "category_id": 0,
                    "segmentation": sorted_imgs_anns[i]['segmentation'],
                    "bbox": sorted_imgs_anns[i]['bbox']
                }]
            x, y, dx, dy = sorted_imgs_anns[i]['bbox']
            x, y, dx, dy = int(x), int(y), int(dx), int(dy)
            min_x_start, min_y_start, max_x_end, max_y_end = x, y, x+dx, y+dy
        if set_name == "train":
            dataset_dicts = dataset_dicts[int(len(dataset_dicts)*0.15):]
        if set_name == "val":
            dataset_dicts = dataset_dicts[:int(len(dataset_dicts)*0.15)]
    elif set_name == "test":
        img_file_pathes = glob(os.path.join(data_dirs, 'test/*.png'))
        for idx, img_file_path in enumerate(img_file_pathes):
            img = Image.open(img_file_path)
            record = {}
            record["file_name"] = img_file_path
            record["image_id"] = idx
            record['width'], record['height'] = Image.open(img_file_path).size
            record['annotations'] = []
            dataset_dicts.append(record)
    return dataset_dicts

'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
for d in ["train", "val", "test"]:
    DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
    MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
plane_metadata = MetadataCatalog.get("data_detection_train")

### Network

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/object_detection".format(BASE_DIR)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ("data_detection_val",)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.MODEL.MASK_ON = True
# cfg.MODEL.BACKBONE.NAME = "build_resnet_backbone"
# cfg.MODEL.RESNETS.DEPTH = 34
# cfg.MODEL.RESNETS.RES2_OUT_CHANNELS = 64
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
# cfg.SOLVER.STEPS = (20, 10000, 20000)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.INPUT.MIN_SIZE_TRAIN = (800,)

### Training

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)


In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3): 
# dataset_dicts = get_detection_data("train")
# for d in dataset_dicts: 
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(20, 20))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
evaluator = COCOEvaluator("data_detection_train", output_dir="./output/object_detection/")
val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`
